In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob,os
import FMR
from scipy.interpolate import interp1d

In [ ]:
directory = '../ppblsmo20A-OOP-redo-highsens/'
olddir = os.getcwd()
os.chdir(directory)
filenames = glob.glob('*.csv')
for f in filenames:
    print f

In [ ]:
def readFile(filename):
    return np.loadtxt(filename, unpack=True, delimiter=',')

Hmeass = []
dPdHs = []
phases = []
freqs = []
for filename in filenames:
    print filename
    Hmeas, dPdH, phase = FMR.readFile(filename)
    Hmeass.append(Hmeas)
    dPdHs.append(dPdH)
    phases.append(phase)
    freq = int(filename.partition('MHz')[0].rpartition('-')[-1])/1000.
    freqs.append(freq)

#os.chdir(olddir)

Convert H_measured to H_sample --> the errors on this measurement and conversion are tiny so we will ignore them

In [ ]:
gaussmeter = np.load('../FieldCalibration/fieldcalib190220.npz')
gaussfit = interp1d(gaussmeter['twosupply_measuredH'], gaussmeter['twosupply_actualH'],kind='cubic')
print 'Hreadscale = '+str(gaussmeter['h_read_scale'])
print 'Hreadoffset = '+str(gaussmeter['h_read_offset'])

Hs = []
for Hmeas in Hmeass:
    #field recorded in the file is field_measured*h_read_scale + h_read_offset - instead we are correcting this to a better calibration from measured -> actual H
    Hraw = (Hmeas-gaussmeter['h_read_offset'])/gaussmeter['h_read_scale']
    Hs.append(gaussfit(Hraw))

In [ ]:
%matplotlib inline
for i,H in enumerate(Hs):
    print str(i)+': '+str(freqs[i])
    plt.plot(H,dPdHs[i],label='corrected')
    plt.plot(Hmeass[i],dPdHs[i],label='arbitrary scaling')
    imax = np.argmax(dPdHs[i])
    plt.plot(H[imax],dPdHs[i][imax],'*')
    imin = np.argmin(dPdHs[i])
    plt.plot(H[imin],dPdHs[i][imin],'*')
    plt.legend()
    plt.show()

From above graphs, some of the spectra do not include any resonance peaks, were aborted before all of the data was taken, etc - removing these from the analysis. Need to then rerun all of the data reading as well

In [ ]:
#sample specific
toremove = ['ppblsmo20A-20190221-Hreadscale1.441-Hreadoffset-11.535-sens10uv-100ms-OP-20000MHz-000.0deg-+8dBm3.0OeRMS-0-2.csv',
            'ppblsmo20A-20190221-Hreadscale1.441-Hreadoffset-11.535-sens10uv-100ms-OP-22500MHz-000.0deg-+8dBm3.0OeRMS-0-5.csv',
            'ppblsmo20A-20190221-Hreadscale1.441-Hreadoffset-11.535-sens10uv-100ms-OP-25000MHz-000.0deg-+8dBm3.0OeRMS-0-1.csv']
for t in toremove:
    filenames.remove(t)
for f in filenames:
    print f

The phase actually gives some interesting information as well - also essentially a Lorentzian peak with the same parameters (we measure X and phase so now we have twice the data!) - so we want to "unwrap" it

In [ ]:
#def unwrapPhase(phase):
#    for i in np.arange(len(phase)-1):
#        phiplus1 = phase[i+1]
#        phi = phase[i]
#        diffs = np.array([360., 0., -360.])
#        whichloop = np.argmin(np.abs(phiplus1-phi+diffs)) #compare difference of angles
#        if whichloop != 1:
#            phase[i+1] += diffs[whichloop]
#    return phase

In [ ]:
for i,p in enumerate(phases):
    phases[i] = FMR.unwrapPhase(p)

In [ ]:
%matplotlib inline
for i,H in enumerate(Hs):
    print freqs[i]
    plt.plot(H,phases[i])
    plt.show()

In [ ]:
%matplotlib inline
for i,H in enumerate(Hs):
    print freqs[i]
    #plt.plot(H,phases[i])
    plt.plot(H,-dPdHs[i]*np.tan(phases[i]*np.pi/180.))
    plt.plot(H,dPdHs[i])
    plt.show()

The above graphs show that there are large errors in Y when X crosses 0 - this may not be as useful to incorporate (maybe more useful to measure X, Y from the lockin rather than X, Theta)

In [ ]:
#
#Ys = []
#for i,p in enumerate(phases):
#    X = np.array(dPdHs[i])
#    Y = X*np.tan(np.array(p)*np.pi/180.)
#    Ys.append(Y)

Background subtraction - If necessary, may need to get rid of noisy points to make the background subtraction easier

In [ ]:
from scipy.optimize import curve_fit

def subtractBG(H, dPdH, window=6, BGfunc=FMR.linearBG):
    #need to fix this to work with negative H
    guessmag, guessres, guesswidth = FMR.guesses(H,dPdH)
    bgH = []
    bg = []
    
    upperbound = np.where(H>guessres+guesswidth*window)[0]
    if len(upperbound) > 0:
        bgend = upperbound[-1]
        bgH.extend(H[0:bgend])
        bg.extend(dPdH[0:bgend])
    else:
        bgend = 0

    lowerbound = np.where(H<guessres-guesswidth*window)[0]
    if len(lowerbound) > 0:
        bgstart = lowerbound[0]
        bgH.extend(H[bgstart:])
        bg.extend(dPdH[bgstart:])
    else:
        bgstart = -1
    
    if len(bg) > 1:
        guess = np.array([0, 0])
        fit = curve_fit(BGfunc, bgH, bg, guess)

        #extract noise
        fitBGregion = BGfunc(np.array(bgH), *(fit[0]))
        s2 = np.sum((bg - fitBGregion)**2) / (len(bgH)-2) #-2 b/c 2 points needed to make a line
        errordPdH = np.ones_like(bgH)*np.sqrt(s2)
        
        #because all of the errors are equal, the covariance matrix of the fit is independent of the errors
        
        fitBG = BGfunc(H, *(fit[0]))
        plt.plot(H,dPdH,'.')
        plt.plot(H,fitBG)
        plt.vlines(H[bgend],np.min(dPdH),np.max(dPdH),linestyles='dotted')
        plt.vlines(H[bgstart],np.min(dPdH),np.max(dPdH),linestyles='dotted')
    else:
        fit = (np.array([0.,0.]), np.array([[0.,0.],[0.,0.]]))
    
    return fit, s2

In [ ]:
curve_fit?

In [ ]:
reload(FMR)

In [ ]:
%matplotlib inline
dPdHoffsets = []
VdPdHoffsets = []
windows = [12,12,12,4,12,12,8,12,12] #adjusted visually
for i,H in enumerate(Hs):
    dPdH = dPdHs[i]
    fit, s2dPdH = subtractBG(H,dPdH,window=windows[i])
    print fit
    print s2dPdH
    print freqs[i]
    plt.show()
    dPdHoffset = dPdH - FMR.linearBG(H,*(fit[0]))
    dPdHoffsets.append(dPdHoffset)
    
    V = np.array([[j*h*fit[1][0][0] + fit[1][1][1] + (j+h)*fit[1][0][1] for j in H] for h in H])
    V2 = np.identity(len(H))*s2dPdH
    VdPdHoffsets.append(V+V2)

In [ ]:
len(V)

In [ ]:
i=26
H = Hs[i]
dPdH = dPdHs[i]
fitBG = FMR.subtractBG(H,dPdH,window=10)
print freqs[i]
plt.show()
dPdHoffset = dPdH - fitBG
dPdHoffsets.append(dPdHoffset)

In [ ]:
reload(FMR)

Fit to the sum of multiple Lorentzian-derivative lineshapes (containing both symmetric and antisymmetric parts), taking into account covariance between different measurements of power - currently I'm only fitting dP/dH (X from the lockin) but in the future I may also use the second measurement, Theta or Y from the lock-in

In [ ]:
def fitFMRcov(H, dPdHoffset, V, guess, debug=False, guessplt=1):
    fit = curve_fit(lambda x,*guess: FMR.LorentzianDerivativeNWrapper(x,guess), H, dPdHoffset, guess, sigma=V)
    fitY = FMR.LorentzianDerivativeNWrapper(H, fit[0])
    
    #splitting parameters into the different N curves
    fitsep = np.array(fit[0])
    fitsep.shape = (fitsep.shape[0]/4,4)

    covsep = []
    for i in np.arange(0,fitsep.shape[0]):
        covpart = fit[1][i*4:(i+1)*4,i*4:(i+1)*4]
        covsep.append(covpart)
    covsep = np.array(covsep)

    covfull = fit[1]

    fitsepY = [FMR.LorentzianDerivative(H, *(f)) for f in fitsep]
    
    plt.plot(H,dPdHoffset,'.')
    if debug:
        plt.plot(H,FMR.LorentzianDerivativeNWrapper(H, guess)*guessplt)
    plt.plot(H,fitY,linewidth=3.0)
    residual = dPdHoffset-fitY
    plt.plot(H,residual+(np.min(fitY)-np.max(residual))*1.5,'.-k')
    for f in fitsepY:
        plt.plot(H,f)
    plt.ylabel('Derivative of Absorbed Power (a.u.)')
    plt.xlabel('External Field (Oe)')
        
    return fitsep,covsep,covfull

In [ ]:
curve_fit?

In [ ]:
#NPeaks = [2]*len(freqs)
#guessPos = ['Left']*len(freqs)
#widths=[3]*len(freqs)
NPeaks = [2, 2, 2, 2, 2, 2, 2, 3, 2]
widths = [6, 6, 6, 2, 6, 6, 4, 6, 6]
guessPos = ['Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Center', 'Left']
fits = []
goodnessoffit = []
%matplotlib inline
for i,H in enumerate(Hs):
    dPdHoffset = dPdHoffsets[i]
    V = VdPdHoffsets[i]
    guess = FMR.guessN(H,dPdHoffset,NPeaks[i],width=widths[i],pos=guessPos[i])
    #if i == 5:
    #    guess[5] = 2*guess[1] - guess[5]
    fit = fitFMRcov(H,dPdHoffset,V,guess)#,debug='True')
    fits.append(fit)
    print str(i)+': '+str(freqs[i])
    
    df = len(H) - fit[0].size
    
    mu = FMR.LorentzianDerivativeNWrapper(H,fit[0].reshape(fit[0].size))
    diff = dPdHoffsets[i] - mu
    chi2 = np.sum(np.matmul(np.matmul(diff.transpose(),np.linalg.inv(V)),diff))
    gf = chi2/df
    goodnessoffit.append(gf)
    print 'goodness of fit: '+str(gf)+', p = '
    print st.chi2.cdf(chi2,df)
    
    plt.show()

In [ ]:
st.chi2.cdf(chi2,df)

In [ ]:
fits[0][0].size

In [ ]:
%matplotlib inline
i = -2
print guessPos[i]
print NPeaks[i]
print widths[i]
print str(i)+': '+str(freqs[i])
dPdHoffset = dPdHoffsets[i]
#guess = FMR.guessN(Hs[i],dPdHoffset,1,width=5,pos='Left')
#fit = FMR.fitFMR(Hs[i],dPdHoffset,guess,debug=True)
#plt.show()
V = VdPdHoffsets[i]
guess = FMR.guessN(Hs[i],dPdHoffset,2,width=6,pos='Left')
fit = fitFMRcov(Hs[i],dPdHoffset,V,guess,debug=True)
p1 = fit[0][0]
mu = FMR.LorentzianDerivativeNWrapper(Hs[i],fit[0].reshape(fit[0].size))
diff = dPdHoffsets[i] - mu
chi2 = np.sum(np.matmul(np.matmul(diff.transpose(),np.linalg.inv(V)),diff))
gf = chi2/df
print 'goodness of fit: '+str(gf)+', p = '
print st.chi2.cdf(chi2,df)
plt.show()
guess = FMR.guessN(Hs[i],dPdHoffset,3,width=6,pos='Left')
fit = fitFMRcov(Hs[i],dPdHoffset,V,guess,debug=True)
p2 = fit[0][-1]
mu = FMR.LorentzianDerivativeNWrapper(Hs[i],fit[0].reshape(fit[0].size))
diff = dPdHoffsets[i] - mu
chi2 = np.sum(np.matmul(np.matmul(diff.transpose(),np.linalg.inv(V)),diff))
gf = chi2/df
print 'goodness of fit: '+str(gf)+', p = '
print st.chi2.cdf(chi2,df)
plt.show()

guess = np.array([p1,p2]).reshape(8)
fit = fitFMRcov(Hs[i],dPdHoffset,V,guess,debug=True)
mu = FMR.LorentzianDerivativeNWrapper(Hs[i],fit[0].reshape(fit[0].size))
plt.plot(Hs[i],dPdHoffsets[i] - FMR.LorentzianDerivativeNWrapper(Hs[i],guess)-0.0000045,'.-')
diff = dPdHoffsets[i] - mu
chi2 = np.sum(np.matmul(np.matmul(diff.transpose(),np.linalg.inv(V)),diff))
gf = chi2/df
print 'goodness of fit: '+str(gf)+', p = '
print st.chi2.cdf(chi2,df)
plt.show()
#print fit[0]

In [ ]:
i = 0
mainpeakparams = fits[i][0][0] #ith frequency, 0=parameters, 0=first peak

print 'Frequency '+str(freqs[i])+' GHz'
print 'Resonant Field '+str(mainpeakparams[1])+' Oe pm '+str(fits[i][1][0][1][1])
print 'Linewidth '+str(mainpeakparams[3])+' Oe pm '+str(fits[i][1][0][3][3])

Hres = np.linspace(mainpeakparams[1]-1., mainpeakparams[1]+1.,100)
delH = np.linspace(mainpeakparams[3]-1.,mainpeakparams[3]+1.,100)

Vinv = np.linalg.inv(VdPdHoffsets[i])

chi2array = []
for hr in Hres:
    row = []
    for d in delH:
        params = fits[i][0].copy()
        params[0][1] = hr
        params[0][3] = d
        params = params.reshape(params.shape[0]*4)
        #print params
        
        mu = FMR.LorentzianDerivativeNWrapper(Hs[i],params)
        diff = dPdHoffsets[i] - mu
        if d == delH[0] and hr == Hres[0]:
            #print params.shape
            %matplotlib inline
            plt.plot(Hs[i],mu)
            plt.plot(Hs[i],diff)
        chi2 = np.sum(np.matmul(np.matmul(diff.transpose(),Vinv),diff))
        row.append(chi2)
    chi2array.append(row)

In [ ]:
chi2array = np.array(chi2array)

import scipy.stats as st
CLs = [0.68, 0.95, 0.99]
NPeak = fits[i][0].shape[0]
q = st.chi2.ppf(CLs,NPeak*4)
print q

ind = np.unravel_index(np.argmin(chi2array, axis=None), chi2array.shape)

m = np.min(chi2array)
print m
print chi2array[ind]
levels = m+np.array(q)
    
%matplotlib inline

fig1, ax1 = plt.subplots()

# Basic contour plot
CS1 = ax1.contour(Hres,delH,chi2array,levels)
ax1.plot(mainpeakparams[1],mainpeakparams[3],'o')
ax1.plot(Hres[ind[1]],delH[ind[0]],'*')

fmt = {}
for l, s in zip(CS1.levels, [str(c*100.)+'%' for c in CLs]):
    fmt[l] = s

# Label every other level using strings
ax1.clabel(CS1, CS1.levels, inline=True, fmt=fmt)#, fontsize=10)
plt.xlabel(r'$H_{FMR}$ for peak 1, Oe')
plt.ylabel(r'$\Delta H$ for peak 1, Oe')
plt.title(r'$\chi^2$ Contour plot')

#s = allparams.shape
#for l in np.arange(0,s[0]):
#    for m in np.arange(0,s[1]):
        

#print allparams.shape
#model = FMR.LorentzianDerivativeNWrapper(Hs[i],allparams)
#making a triangle plot

print 'rho = '+str(fits[i][2][1][3]/np.sqrt(fits[i][2][1][1]*fits[i][2][3][3]))

In [ ]:
i = 7
mainpeakparams = fits[i][0][0] #ith frequency, 0=parameters, 0=first peak

print 'Frequency '+str(freqs[i])+' GHz'
print 'Resonant Field '+str(mainpeakparams[1])+' Oe pm '+str(fits[i][1][0][1][1])
print 'Linewidth '+str(mainpeakparams[3])+' Oe pm '+str(fits[i][1][0][3][3])

Hres = np.linspace(mainpeakparams[1]-10., mainpeakparams[1]+10.,100)
delH = np.linspace(mainpeakparams[3]-10.,mainpeakparams[3]+10.,100)

Vinv = np.linalg.inv(VdPdHoffsets[i])

chi2array = []
for hr in Hres:
    row = []
    for d in delH:
        params = fits[i][0].copy()
        params[0][1] = hr
        params[0][3] = d
        params = params.reshape(params.shape[0]*4)
        #print params
        
        mu = FMR.LorentzianDerivativeNWrapper(Hs[i],params)
        diff = dPdHoffsets[i] - mu
        if d == delH[0] and hr == Hres[0]:
            #print params.shape
            %matplotlib inline
            plt.plot(Hs[i],mu)
            plt.plot(Hs[i],diff)
        chi2 = np.sum(np.matmul(np.matmul(diff.transpose(),Vinv),diff))
        row.append(chi2)
    chi2array.append(row)

In [ ]:
chi2array = np.array(chi2array)

import scipy.stats as st
CLs = [0.68, 0.95, 0.99]
NPeak = fits[i][0].shape[0]
q = st.chi2.ppf(CLs,NPeak*4)
print q

ind = np.unravel_index(np.argmin(chi2array, axis=None), chi2array.shape)

m = np.min(chi2array)
print m
print chi2array[ind]
levels = m+np.array(q)
    
%matplotlib inline

fig1, ax1 = plt.subplots()

# Basic contour plot
CS1 = ax1.contour(Hres,delH,chi2array,levels)
ax1.plot(mainpeakparams[1],mainpeakparams[3],'o')
ax1.plot(Hres[ind[1]],delH[ind[0]],'*')

fmt = {}
for l, s in zip(CS1.levels, [str(c*100.)+'%' for c in CLs]):
    fmt[l] = s

# Label every other level using strings
ax1.clabel(CS1, CS1.levels, inline=True, fmt=fmt)#, fontsize=10)
plt.xlabel(r'$H_{FMR}$ for peak 1, Oe')
plt.ylabel(r'$\Delta H$ for peak 1, Oe')
plt.title(r'$\chi^2$ Contour plot')

#s = allparams.shape
#for l in np.arange(0,s[0]):
#    for m in np.arange(0,s[1]):
        

#print allparams.shape
#model = FMR.LorentzianDerivativeNWrapper(Hs[i],allparams)
#making a triangle plot

print 'rho = '+str(fits[i][2][1][3]/np.sqrt(fits[i][2][1][1]*fits[i][2][3][3]))

Gaussian! But what about the other fit parameters?

In [ ]:
i = 7
peakparams = fits[i][0] #ith frequency, 0=parameters, 0=first peak

print 'Frequency '+str(freqs[i])+' GHz'
print 'Linewidth 1 '+str(peakparams[0][3])+' Oe pm '+str(fits[i][1][0][3][3])
print 'Linewidth 2 '+str(peakparams[-1][3])+' Oe pm '+str(fits[i][1][-1][3][3])

delH1 = np.linspace(peakparams[0][3]-10., peakparams[0][3]+10.,100)
delH2 = np.linspace(peakparams[-1][3]-10.,peakparams[-1][3]+10.,100)

Vinv = np.linalg.inv(VdPdHoffsets[i])

chi2array = []
for d1 in delH1:
    row = []
    for d2 in delH2:
        params = fits[i][0].copy()
        params[0][3] = d1
        params[-1][3] = d2
        params = params.reshape(params.shape[0]*4)
        #print params
        
        mu = FMR.LorentzianDerivativeNWrapper(Hs[i],params)
        diff = dPdHoffsets[i] - mu
        if d1 == delH1[0] and d2 == delH2[0]:
            #print params.shape
            %matplotlib inline
            plt.plot(Hs[i],mu)
            plt.plot(Hs[i],diff)
        chi2 = np.sum(np.matmul(np.matmul(diff.transpose(),Vinv),diff))
        row.append(chi2)
    chi2array.append(row)

In [ ]:
chi2array = np.array(chi2array)

import scipy.stats as st
CLs = [0.68, 0.95, 0.99]
NPeak = fits[i][0].shape[0]
q = st.chi2.ppf(CLs,NPeak*4)
print q

ind = np.unravel_index(np.argmin(chi2array, axis=None), chi2array.shape)

m = np.min(chi2array)
print m
print chi2array[ind]
levels = m+np.array(q)
    
%matplotlib inline

fig1, ax1 = plt.subplots()

# Basic contour plot
CS1 = ax1.contour(delH1,delH2,chi2array,levels)
ax1.plot(peakparams[0][3],peakparams[-1][3],'o')
ax1.plot(delH1[ind[1]],delH2[ind[0]],'*')

fmt = {}
for l, s in zip(CS1.levels, [str(c*100.)+'%' for c in CLs]):
    fmt[l] = s

# Label every other level using strings
ax1.clabel(CS1, CS1.levels, inline=True, fmt=fmt)#, fontsize=10)
plt.xlabel(r'$\Delta H$ for peak 1, Oe')
plt.ylabel(r'$\Delta H$ for peak 3, Oe')
plt.title(r'$\chi^2$ Contour plot')

#s = allparams.shape
#for l in np.arange(0,s[0]):
#    for m in np.arange(0,s[1]):
        

#print allparams.shape
#model = FMR.LorentzianDerivativeNWrapper(Hs[i],allparams)
#making a triangle plot

print 'rho = '+str(fits[i][2][3][11]/np.sqrt(fits[i][2][3][3]*fits[i][2][11][11]))

In [ ]:
#plot for linewidths for different peaks

i = 0
peakparams = fits[i][0] #ith frequency, 0=parameters, 0=first peak

print 'Frequency '+str(freqs[i])+' GHz'
print 'Linewidth 1 '+str(peakparams[0][3])+' Oe pm '+str(np.sqrt(fits[i][2][3][3]))
print 'Linewidth 2 '+str(peakparams[1][3])+' Oe pm '+str(np.sqrt(fits[i][2][7][7]))

delH1 = np.linspace(peakparams[0][3]-3., peakparams[0][3]+3.,300)
delH2 = np.linspace(peakparams[1][3]-3., peakparams[1][3]+3.,300)

Vinv = np.linalg.inv(VdPdHoffsets[i])

chi2array = []
for d1 in delH1:
    row = []
    for d2 in delH2:
        params = fits[i][0].copy()
        params[0][3] = d1
        params[1][3] = d2
        params = params.reshape(params.shape[0]*4)
        #print params
        
        mu = FMR.LorentzianDerivativeNWrapper(Hs[i],params)
        diff = dPdHoffsets[i] - mu
        chi2 = np.sum(np.matmul(np.matmul(diff.transpose(),Vinv),diff))
        row.append(chi2)
    chi2array.append(row)

In [ ]:
print 'Frequency '+str(freqs[i])+' GHz'
print 'Linewidth 1 '+str(peakparams[0][3])+' Oe pm '+str(np.sqrt(fits[i][1][0][3][3]))
print 'Linewidth 2 '+str(peakparams[1][3])+' Oe pm '+str(np.sqrt(fits[i][1][1][3][3]))

In [ ]:
chi2array = np.array(chi2array)

import scipy.stats as st
CLs = [0.68, 0.95, 0.99]
#CLs = [0.5, 0.68, 0.8, 0.95, 0.99, 0.999]
NPeak = fits[i][0].shape[0]
q = st.chi2.ppf(CLs,NPeak*4)
print q

ind = np.unravel_index(np.argmin(chi2array, axis=None), chi2array.shape)

m = np.min(chi2array)
print m
print chi2array[ind]
levels = m+np.array(q)
    
%matplotlib inline

fig1, ax1 = plt.subplots()

# Basic contour plot
CS1 = ax1.contour(delH1,delH2,chi2array,levels)
ax1.plot(peakparams[0][3],peakparams[1][3],'o')
ax1.plot(delH1[ind[1]],delH2[ind[0]],'*')

fmt = {}
for l, s in zip(CS1.levels, [str(c*100.)+'%' for c in CLs]):
    fmt[l] = s

# Label every other level using strings
ax1.clabel(CS1, CS1.levels, inline=True, fmt=fmt)#, fontsize=10)
ax1.set_aspect('equal')
plt.xlabel(r'$\Delta H$ for peak 1, Oe')
plt.ylabel(r'$\Delta H$ for peak 2, Oe')
plt.title(r'$\chi^2$ Contour plot')

#s = allparams.shape
#for l in np.arange(0,s[0]):
#    for m in np.arange(0,s[1]):
        

#print allparams.shape
#model = FMR.LorentzianDerivativeNWrapper(Hs[i],allparams)
#making a triangle plot

print 'rho = '+str(fits[i][2][3][7]/np.sqrt(fits[i][2][3][3]*fits[i][2][7][7]))

In [ ]:
outfile = 'ppblsmo20A-20190221-peakfits'
np.savez(outfile, frequenciesGHz = freqs, spectrumfieldOe = Hs, spectrumpowerau = dPdHoffsets,
         spectrumphasedeg = phases, spectrumpowercovariance = VdPdHoffsets,
         LorentzianFitParamsSeparated = [f[0] for f in fits], LorentzianFitCovariancesSeparated = [f[1] for f in fits],
         LorentzianFitCovarianceFull = [f[2] for f in fits])
         
         #resonantFieldOe = HFMR_mainpeak, linewidthOe = deltaH_mainpeak,
         #resonantFieldErrorOe = HFMR_err_main, linewidthErrorOe = deltaH_err_main,
         #peakPhasedeg = phase_mainpeak, peakMagnitude = amp_mainpeak,
         #MeffectiveOe = Meff, gilbertDamping = alpha, orbitalgop = gop, intrinsiclinewidthOe = deltaH0,
         #resonantFieldFitCoefficients = HFMRfit, linewidthFitCoefficients = deltaHfit)
#insert CRO thickness, LSMO thickness into this file??

In [ ]:
HFMR_mainpeak = []
HFMR_sidepeak = []
amp_mainpeak = []
amp_sidepeak = []
phase_mainpeak = []
phase_sidepeak = []
deltaH_mainpeak = []
deltaH_sidepeak = []
freqs_sidepeak = []
freqs_peak3 = []
HFMR_peak3 = []
deltaH_peak3 = []
amp_peak3 = []
phase_peak3 = []
for i,f in enumerate(fits):
    if NPeaks[i] != 1:
        side = -1
        freqs_sidepeak.append(freqs[i])
        HFMR_sidepeak.append(f[0][side][1])
        deltaH_sidepeak.append(f[0][side][3])
        amp_sidepeak.append(f[0][side][0])
        phase_sidepeak.append(f[0][side][2])
    peak = -2
    HFMR_mainpeak.append(f[0][peak][1])
    deltaH_mainpeak.append(f[0][peak][3])
    amp_mainpeak.append(f[0][peak][0])
    phase_mainpeak.append(f[0][peak][2])
    if NPeaks[i] == 3:
        p3 = -3
        freqs_peak3.append(freqs[i])
        HFMR_peak3.append(f[0][p3][1])
        deltaH_peak3.append(f[0][p3][3])
        amp_peak3.append(f[0][p3][0])
        phase_peak3.append(f[0][p3][2])

In [ ]:
plt.plot(freqs,HFMR_mainpeak,'.')
plt.plot(freqs_sidepeak,HFMR_sidepeak,'.')
plt.plot(freqs_peak3,HFMR_peak3,'.')
plt.show()
plt.plot(freqs,deltaH_mainpeak,'.')
plt.plot(freqs_sidepeak,deltaH_sidepeak,'.')
plt.plot(freqs_peak3,deltaH_peak3,'.')
plt.show()
plt.plot(freqs,amp_mainpeak,'.')
plt.plot(freqs_sidepeak,amp_sidepeak,'.')
plt.plot(freqs_peak3,amp_peak3,'.')
plt.show()
plt.plot(freqs,phase_mainpeak,'.')
plt.plot(freqs_sidepeak,phase_sidepeak,'.')
plt.plot(freqs_peak3,phase_peak3,'.')
plt.show()

In [ ]:
freqs_sidepeak = []
HFMR_mainpeak = []
HFMR_sidepeak = []
HFMR_err_main = []
HFMR_err_side = []
amp_mainpeak = []
amp_sidepeak = []
phase_mainpeak = []
phase_sidepeak = []
deltaH_mainpeak = []
deltaH_sidepeak = []
deltaH_err_main = []
deltaH_err_side = []
for i,f in enumerate(fits):
    if NPeaks[i] == 2:
        amps = [a[0] for a in f[0]]
        peak = np.argmax(amps)
        
        side = np.argmin(amps)
        
        peak=0
        
        HFMR_sidepeak.append(f[0][side][1])
        HFMR_err_side.append(f[1][side][1])
        deltaH_sidepeak.append(f[0][side][3])
        deltaH_err_side.append(f[1][side][3])
        amp_sidepeak.append(f[0][side][0])
        phase_sidepeak.append(f[0][side][2])
        freqs_sidepeak.append(freqs[i])
    elif NPeaks[i] == 3:
        amps = [a[0] for a in f[0]]
        peak = np.argmax(amps)
        
        side = np.argmin(amps)
        peak = 0
        
        HFMR_sidepeak.append(f[0][side][1])
        HFMR_err_side.append(f[1][side][1])
        deltaH_sidepeak.append(f[0][side][3])
        deltaH_err_side.append(f[1][side][3])
        amp_sidepeak.append(f[0][side][0])
        phase_sidepeak.append(f[0][side][2])
        freqs_sidepeak.append(freqs[i])
    elif NPeaks[i] == 1:
        peak = 0
    HFMR_mainpeak.append(f[0][peak][1])
    HFMR_err_main.append(f[1][peak][1])
    deltaH_mainpeak.append(f[0][peak][3])
    deltaH_err_main.append(f[1][peak][3])
    amp_mainpeak.append(f[0][peak][0])
    phase_mainpeak.append(f[0][peak][2])

In [ ]:
plt.plot(freqs,HFMR_mainpeak,'.',label='max peak')
plt.plot(freqs_sidepeak,HFMR_sidepeak,'.',label='side peak')
plt.legend()
plt.show()
plt.plot(freqs,deltaH_mainpeak,'.',label='max peak')
plt.plot(freqs_sidepeak,deltaH_sidepeak,'.',label='side peak')
plt.legend()
plt.show()
plt.plot(freqs,amp_mainpeak,'.',label='max peak')
plt.plot(freqs_sidepeak,amp_sidepeak,'.',label='side peak')
plt.legend()
plt.show()
plt.plot(freqs,phase_mainpeak,'.',label='max peak')
plt.plot(freqs_sidepeak,phase_sidepeak,'.',label='side peak')
plt.legend()
plt.show()

# TODO: measure saturation field from SQUID

In [ ]:
#need to take into account saturation magnetization
freqsort = np.argsort(np.array(freqs))

freqslice = slice(None,None)
HFMRfit = np.polyfit(np.array(HFMR_mainpeak)[freqsort][freqslice],np.array(freqs)[freqsort][freqslice],1)
print HFMRfit
Meff = -HFMRfit[1]/(HFMRfit[0]*4*np.pi)

print 'Meff: '+str(Meff)+' emu/cm^3'

h = 6.62607004e-34*1e7 #erg-s
muB = 9.274000999e-21 #erg/G
mu0 = 1.00000037 #G/Oe - https://www.engineeringtoolbox.com/permeability-d_1923.html
giga = 1e9
gop = HFMRfit[0]*h/(muB*mu0)*giga
print 'gop: '+str(gop)

%matplotlib inline
plt.errorbar(freqs,HFMR_mainpeak,fmt='.',yerr=HFMR_err_main)
plt.errorbar(np.array(freqs)[freqsort][freqslice],np.array(HFMR_mainpeak)[freqsort][freqslice],fmt='.',yerr=np.array(HFMR_err_main)[freqsort][freqslice])
plt.plot(np.poly1d(HFMRfit)(HFMR_mainpeak),HFMR_mainpeak,'-',label='corrected')
plt.legend()
plt.ylabel(r'H$_{FMR}$ (Oe)')
plt.xlabel('Frequency (GHz)')

In [ ]:
freqslice = slice(None,None)
deltaH = deltaH_mainpeak
errdeltaH = deltaH_err_main

deltaHfit = np.polyfit(np.array(freqs)[freqsort][freqslice],np.array(deltaH)[freqsort][freqslice],1,w=1./np.array(errdeltaH)[freqsort][freqslice])
print deltaHfit

deltaH0 = deltaHfit[1]
print "deltaH_0: "+str(deltaH0)+' Oe'

OetoAperm = 1000/(4*np.pi)

alpha = deltaHfit[0]*HFMRfit[0]

print "alpha: "+str(alpha)

In [ ]:
%matplotlib inline
plt.errorbar(freqs,deltaH,fmt='.',yerr=errdeltaH, label='corrected')
plt.errorbar(np.array(freqs)[freqsort][freqslice],np.array(deltaH)[freqsort][freqslice],fmt='.',yerr=np.array(errdeltaH)[freqsort][freqslice], label='corrected')
plt.plot(freqs,np.poly1d(deltaHfit)(freqs),'-',label='corrected')
plt.legend()
plt.ylabel('FMR Linewidth (Oe)')
plt.xlabel('Frequency (GHz)')

In [ ]:
outfile = 'ppblsmo20A-20190221-sens10uv-100ms-OP-+8dBm3.0OeRMS'
np.savez(outfile, frequenciesGHz = freqs, resonantFieldOe = HFMR_mainpeak, linewidthOe = deltaH_mainpeak,
         resonantFieldErrorOe = HFMR_err_main, linewidthErrorOe = deltaH_err_main,
         peakPhasedeg = phase_mainpeak, peakMagnitude = amp_mainpeak,
         MeffectiveOe = Meff, gilbertDamping = alpha, orbitalgop = gop, intrinsiclinewidthOe = deltaH0,
         resonantFieldFitCoefficients = HFMRfit, linewidthFitCoefficients = deltaHfit)
#insert CRO thickness, LSMO thickness into this file??